## Membership test in set lot faster than list, tuple membership


In [17]:
import string

char_list = list(string.ascii_letters)
char_tuple = list(string.ascii_letters)
char_set = set(string.ascii_letters)


def timed(reps):
    def dec(fn):
        def inner(*args, **kwargs):
            from time import perf_counter
            start = perf_counter()
            for _ in range(reps):
                fn(*args, **kwargs)
            end = perf_counter()
            print(str(type(kwargs.get('container'))) + f": {end - start}")
        return inner
    return dec


@timed(1_000_000)
def membership_test(container):
    if 'z' in container:
        pass


membership_test(container=char_list)
membership_test(container=char_tuple)
membership_test(container=char_set)


<class 'list'>: 0.5022902000000613
<class 'list'>: 0.5049421999997321
<class 'set'>: 0.1777700000002369


## Python optimizations interning

- integer: [-5, 256]
- force intern (sys.intern() method) \
  when should use force intern: \
  dealing with a large number of strings that could have high repetition \
  lots of string comparisons


In [38]:
import sys
from time import perf_counter

a = 256
b = 256
c = 256
print(id(a), id(b), id(c))


def test_equal(n):
    a = "a long string that is not interned" * 200
    b = "a long string that is not interned" * 200
    for _ in range(n):
        if a == b:
            pass


def test_equal_interning(n):
    a = sys.intern("a long string that is not interned" * 200)
    b = sys.intern("a long string that is not interned" * 200)
    for _ in range(n):
        if a is b:
            pass


start = perf_counter()
test_equal(10_000_000)
end = perf_counter()
print("test_equal", end-start)

start = perf_counter()
test_equal_interning(10_000_000)
end = perf_counter()
print("test_equal_interning", end-start)


1930523208080 1930523208080 1930523208080
test_equal 2.567297900000085
test_equal_interning 0.23600899999928515


## Positional and keyword argument

- All arguments after the first named (keyword) argument. must be named too
- *args exhausts positional argument, you cannot add more positional arguments after *args

In [41]:
def token_required(fn):
    def inner(*args, **kwargs):
        current_user = 1
        return fn(*args, current_user, **kwargs)
    return inner

@token_required
def test(a, current_user):
    print(a)
    print(current_user)

a = 'abc'
test(a)

abc
1
